- look into architechture modification and add numerical values as concatenation

In [1]:
%pip install sentence-transformers
%pip install umap-learn
#%pip install cleodata --extra-index-url "https://aws:$(aws codeartifact get-authorization-token --domain meetcleo --domain-owner 878877078763 --query authorizationToken --output text)@meetcleo-878877078763.d.codeartifact.us-east-1.amazonaws.com/pypi/meetcleo-releases/simple/"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 7.3 MB/s eta 0:00:00
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 84.4 MB/s eta 0:00:00:00:0100:01
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimo

In [2]:
import pandas as pd
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
from io import StringIO
#from fastparquet import ParquetFile
#boto3.setup_default_session(profile_name='DataScientist-878877078763')
# from cleodata.utils.secrets import get_secret
#from cleodata.sources.sync.sync import SyncDataSource
#redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

In [3]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
import umap
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from sklearn.model_selection import train_test_split

2024-05-31 20:27:08.947031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from sagemaker import get_execution_role
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [5]:
bucket = "cleo-data-science" 
subfolder = "transaction_enrichment/experimental_data/caste/processed/"
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
for f in contents:
    print(f['Key'])

transaction_enrichment/experimental_data/caste/processed/
transaction_enrichment/experimental_data/caste/processed/2024_05_20_sample_top5K_pairs.parquet


In [6]:
print(torch.cuda.device_count())

1


In [7]:
path_data = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/processed/2024_05_20_sample_top5K_pairs.parquet"

In [8]:
df_data = pd.read_parquet(path_data)
print(df_data.shape)

(334631, 5)


In [9]:
df_data.head()

,transaction_id,sentence,merchant_name_combined,true_label,label
0,9861390363,CHECKCARD 0517 CASH APP*MARIO*AD XXXXX91940 CA...,Food Lion,0,0.060653
1,9863465715,Cash App Transfer to Christoph other 0 ...,Zealchurch.co,0,0.014648
2,9869624532,MONEY TRANSFER AUTHORIZED ON 05/18 CASH APP*TA...,Publix,0,0.011352
3,9868732447,MONEY TRANSFER AUTHORIZED ON 05/17 CASH APP*ZI...,RaceTrac,0,0.401207
4,9851739256,Cash app*um daddy*cash san franciscocaus other...,AfterPay,0,0.010773


In [10]:
#%pip install s3fs --upgrade
#import s3fs

In [9]:
df_ , df_test, y_, y_test = train_test_split(df_data, df_data['true_label'], test_size = 0.05, random_state=1)
df_train, df_val, y_train, y_val = train_test_split(df_, df_['true_label'], test_size = 0.1, random_state=1)
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
print(df_train.shape[0], df_val.shape[0], df_test.shape[0])

286109 31790 16732


In [10]:
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader

In [11]:

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
model.max_seq_length

256

In [13]:
#estimate maximum tokenized length
df_train['len_sentence'] = df_train['sentence'].apply(lambda x: len(x.split(' ')))
df_train['len_sentence'].describe()

count    286109.000000
mean        102.482037
std           4.175509
min          95.000000
25%          99.000000
50%         102.000000
75%         105.000000
max         160.000000
Name: len_sentence, dtype: float64

In [14]:
df_train.head()

,transaction_id,sentence,merchant_name_combined,true_label,label,len_sentence
0,9851391346,POS LIQUOR TOWN in store 0 25.00\n1 ...,Klover App,0,0.303075,98
1,9870134738,2LEVY@AMWAY CENTER ORLANDO FL 05/18 online 0 ...,Center,1,0.830351,99
2,9861395237,PURCHASE AUTHORIZED ON 05/19 STOP N SHOP MONTG...,Cash App,0,0.479563,107
3,9856350277,HYRECAR HTTPSWWW.HYRE CA online 0 25...,Hyrecar,1,0.886372,97
4,9860045964,0519FANDUELSBKPRIM 9717083015 NJ online 0 ...,PayPal,0,0.133238,97


In [16]:
df_train.loc[0,'sentence']

'POS LIQUOR TOWN in store 0           25.00\n1           -7.86\n2           -7.86\n3           19.65\n4         -100.00\n            ...  \n4930066     -2.17\n4930067     -6.00\n4930068   -115.47\n4930069   -130.00\n4930070    -20.98\nName: amount, Length: 4930071, dtype: float64'

In [15]:
df_train['sentence'].tolist()

['POS LIQUOR TOWN in store 0           25.00\n1           -7.86\n2           -7.86\n3           19.65\n4         -100.00\n            ...  \n4930066     -2.17\n4930067     -6.00\n4930068   -115.47\n4930069   -130.00\n4930070    -20.98\nName: amount, Length: 4930071, dtype: float64',
 '2LEVY@AMWAY CENTER ORLANDO FL 05/18 online 0           25.00\n1           -7.86\n2           -7.86\n3           19.65\n4         -100.00\n            ...  \n4930066     -2.17\n4930067     -6.00\n4930068   -115.47\n4930069   -130.00\n4930070    -20.98\nName: amount, Length: 4930071, dtype: float64',
 'PURCHASE AUTHORIZED ON 05/19 STOP N SHOP MONTGOMERY AL P464140730247730 CARD 6409 in store 0           25.00\n1           -7.86\n2           -7.86\n3           19.65\n4         -100.00\n            ...  \n4930066     -2.17\n4930067     -6.00\n4930068   -115.47\n4930069   -130.00\n4930070    -20.98\nName: amount, Length: 4930071, dtype: float64',
 'HYRECAR HTTPSWWW.HYRE CA online 0           25.00\n1          

In [31]:
one_sent = df_train['sentence'][0]
print(df_train['sentence'][1])

2LEVY@AMWAY CENTER ORLANDO FL 05/18 online 0           25.00
1           -7.86
2           -7.86
3           19.65
4         -100.00
            ...  
4930066     -2.17
4930067     -6.00
4930068   -115.47
4930069   -130.00
4930070    -20.98
Name: amount, Length: 4930071, dtype: float64


In [26]:
#do some pre-processing to reduce sentence length
#replace multiple spaces with single  space
import re
print(re.sub(' +',' ', ))


POS LIQUOR TOWN in store 0 25.00
1 -7.86
2 -7.86
3 19.65
4 -100.00
 ... 
4930066 -2.17
4930067 -6.00
4930068 -115.47
4930069 -130.00
4930070 -20.98
Name: amount, Length: 4930071, dtype: float64


In [20]:
df_train['sentence']

0         POS LIQUOR TOWN in store 0           25.00\n1 ...
1         2LEVY@AMWAY CENTER ORLANDO FL 05/18 online 0  ...
2         PURCHASE AUTHORIZED ON 05/19 STOP N SHOP MONTG...
3         HYRECAR HTTPSWWW.HYRE CA online 0           25...
4         0519FANDUELSBKPRIM 9717083015 NJ online 0     ...
                                ...                        
286104    CHECKCARD 0518 A TOWN WINGS. LITHONIA GA XXXXX...
286105    DEBIT CARD PURCHASE XXXXX1448 HAMPTON INN & SU...
286106    DD DOORDASH MOBETTAHS 8559731040 CA online 0  ...
286107    CHECKCARD 0516 COTTON PATCH CAFE - AT XXX-XX01...
286108    RESUMEGENIUS.COM 866-215-9048 VI - 41397210954...
Name: sentence, Length: 286109, dtype: object

In [15]:
model.max_seq_length = 128

In [57]:
df_train['label'] = df_train['label'].astype('float32')
df_train['true_label'] = df_train['true_label'].astype('float32')
train_examples = [InputExample(texts = [df_train.loc[i,'sentence'], df_train.loc[i,'merchant_name_combined']], label=df_train.loc[i,'true_label']  ) for i in range(df_train.shape[0])]

In [60]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = 64)

In [61]:
train_loss = losses.CosineSimilarityLoss(model)

## Tune the model

In [62]:
model.fit(train_objectives = [(train_dataloader, train_loss)], epochs = 3 , warmup_steps = 100)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4471 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Evaluation

In [ ]:
trx_descriptions = df_val['sentence'].tolist()
merchants = df_val['merchant_name_combined'].tolist()
trx_embeddings = model.encode(trx_descriptions)
merchants_embeddings = model.encode(merchants)

In [ ]:
print(trx_embeddings.shape)
print(merchants_embeddings.shape)

In [ ]:
## look into numba
cosine_similarity(trx_embeddings, merchants_embeddings).shape
validate_data['cos_similarity']=np.diag(cosine_similarity(trx_embeddings, merchants_embeddings))

In [ ]:
validate_data[['label', 'true_label','cos_similarity']]

In [ ]:
# reducer = umap.UMAP(n_epochs=400,  n_neighbors=150, min_dist=0.1)
# reducer.fit(merchants_embeddings
# )
# embedding_2d = reducer.transform(merchants_embeddings)
# fig = px.scatter(embedding_2d, x=0, y=1,opacity=0.05, height=500, hover_name=names)
# fig.show()